## Web Scrapping weather data from www.timeanddate.com

The goal is to gather the daily temperature humidity and pressure from September 2009 to present.

In [ ]:
import pandas as pd

In [ ]:
import requests
import os
import shutil

from bs4 import BeautifulSoup
import json


In [ ]:
url_start = "https://www.timeanddate.com/weather/usa/oakland/historic?month={}&year={}"

#### Test webscraping process


In [ ]:
year = 2023
months = 7

url = url_start.format(month, year)

import_data = requests.get(url)
yearmonth = str(year) + str(month)

with open("test_{}.html".format(yearmonth), "w+") as f:
    f.write(import_data.text)

In [ ]:
with open("test_20237.html") as f:
    page = f.read()

In [ ]:
import re
#page.text
soup = BeautifulSoup(page, 'html.parser')

s =soup.find(class_="headline-banner__wrap")

In [ ]:
type(s)

In [ ]:
d = str(s)
p = re.compile("}],\"detail\":(\[{.*?}\]),", flags=re.M)
m = p.findall(d)[0] # Neeed [0] to remove the list [] created from findall
data = json.loads(m)
print(data[0:5]) 
type(data)

In [ ]:
df=pd.DataFrame.from_dict(data, orient='columns')
df.head()

In [ ]:
col = ['ds', 'temp', 'templow', 'baro', 'wind', 'wd', 'hum', 'desc']
df[col].head()

#### Convert to datetime

In [ ]:
pattern = r"day, (.*?) —"
df['ds'] = df['ds'].astype('string').str.extract(pattern)
df['ds'] = pd.to_datetime(df['ds'] ,infer_datetime_format=True)


In [ ]:
col = ['temp', 'templow', 'baro', 'wind', 'wd', 'hum']
df[col].astype('int')

In [ ]:
df.head()

In [ ]:
df.info()

### Putting it all together

Collect all of the data for Oakland from September 2009 to July 2023

There is time delay inside of the for loop to slow down the webscrapping with the delay the loop take 3 min to run

In [173]:
import time
years = list(range(2009,2024))
months = list(range(1,13))

dfs = [] # list of dataframes

url_start = "https://www.timeanddate.com/weather/usa/oakland/historic?month={}&year={}"
for year in years:
    for month in months:
        if (year ==2009 and month < 9) or (year ==2023 and month > 7):
            pass
            # Skip the values in the for loop before Sept 2009 and after July 2023
        else:
            url = url_start.format(month, year)
            import_data = requests.get(url)
            page = import_data.text

            soup = BeautifulSoup(page, 'html.parser')

            s =soup.find(class_="headline-banner__wrap") # Isolate the class with the data
            d = str(s)
            p = re.compile("}],\"detail\":(\[{.*?}\]),", flags=re.M)
            m = p.findall(d)[0] # Neeed [0] to remove the list [] created from findall
            data = json.loads(m)
            temp_df=pd.DataFrame.from_dict(data, orient='columns') # Convert list of dictionarys to a dataframe
            dfs.append(temp_df) # Make a list of dataframes
            
            time.sleep(1) # slow down the web scraping to keep a reasonable level of internet traffic

##### Combine into single dataframe

In [174]:
df = pd.concat(dfs) # combine the list of dataframes into a single dataframe

df.head()

,hl,hls,hlsh,date,ts,ds,icon,desc,temp,templow,baro,wind,wd,hum
0,True,"Tue, Sep 8",Sep 8,1.252411e+12,12 pm,"Tuesday, September 8, 2009, 12:00 pm — 6:00 pm",1,Sunny.,69.8,69.8,29.85,6.836,300,53
1,NaN,NaN,NaN,1.252433e+12,6 pm,"Tuesday, September 8, 2009, 6:00 pm — 12:00 am",13,Clear.,57.2,57.2,29.89,4.350,330,87
2,True,"Wed, Sep 9",Sep 9,1.252454e+12,12 am,"Wednesday, September 9, 2009, 12:00 am — 6:00 am",13,Clear.,55.4,51.8,29.91,1.243,320,94
3,NaN,NaN,NaN,1.252476e+12,6 am,"Wednesday, September 9, 2009, 6:00 am — 12:00 pm",36,No weather data available,NaN,NaN,0.00,0.000,0,0
4,NaN,NaN,NaN,1.252498e+12,12 pm,"Wednesday, September 9, 2009, 12:00 pm — 6:00 pm",2,Passing clouds.,66.2,64.4,29.97,11.807,300,65


##### Clean the dataframe

In [175]:
df['desc'].astype('string')
col = ['temp', 'templow', 'baro', 'wind', 'wd', 'hum']
df[col].astype('int')   # gives an error because of null values and missing data for those dates
df.head()

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [176]:
pattern = r"day, (.*?) —"
df['ds'] = df['ds'].astype('string').str.extract(pattern)
df['ds'] = pd.to_datetime(df['ds'] ,infer_datetime_format=True)
df.head()

,hl,hls,hlsh,date,ts,ds,icon,desc,temp,templow,baro,wind,wd,hum
0,True,"Tue, Sep 8",Sep 8,1.252411e+12,12 pm,2009-09-08 12:00:00,1,Sunny.,69.8,69.8,29.85,6.836,300,53
1,NaN,NaN,NaN,1.252433e+12,6 pm,2009-09-08 18:00:00,13,Clear.,57.2,57.2,29.89,4.350,330,87
2,True,"Wed, Sep 9",Sep 9,1.252454e+12,12 am,2009-09-09 00:00:00,13,Clear.,55.4,51.8,29.91,1.243,320,94
3,NaN,NaN,NaN,1.252476e+12,6 am,2009-09-09 06:00:00,36,No weather data available,NaN,NaN,0.00,0.000,0,0
4,NaN,NaN,NaN,1.252498e+12,12 pm,2009-09-09 12:00:00,2,Passing clouds.,66.2,64.4,29.97,11.807,300,65


In [177]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20247 entries, 0 to 74
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   hl       5063 non-null   object        
 1   hls      5063 non-null   object        
 2   hlsh     5063 non-null   object        
 3   date     20247 non-null  float64       
 4   ts       20247 non-null  object        
 5   ds       20247 non-null  datetime64[ns]
 6   icon     20247 non-null  int64         
 7   desc     20247 non-null  object        
 8   temp     20155 non-null  float64       
 9   templow  20155 non-null  float64       
 10  baro     20247 non-null  float64       
 11  wind     20247 non-null  float64       
 12  wd       20247 non-null  int64         
 13  hum      20247 non-null  int64         
dtypes: datetime64[ns](1), float64(5), int64(3), object(5)
memory usage: 2.3+ MB


In [178]:
col_keep = ['ds', 'temp', 'templow', 'baro', 'wind', 'wd', 'hum', 'desc']
new_df = df[col_keep]


In [179]:
new_df.rename(columns={"ds": "date"}, inplace=True)
new_df.reset_index(drop=True, inplace=True)
new_df.head()

/var/folders/th/4cfl7z012fg_zjvld08z7qy80000gn/T/ipykernel_88307/1800194360.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.rename(columns={"ds": "date"}, inplace=True)


,date,temp,templow,baro,wind,wd,hum,desc
0,2009-09-08 12:00:00,69.8,69.8,29.85,6.836,300,53,Sunny.
1,2009-09-08 18:00:00,57.2,57.2,29.89,4.350,330,87,Clear.
2,2009-09-09 00:00:00,55.4,51.8,29.91,1.243,320,94,Clear.
3,2009-09-09 06:00:00,NaN,NaN,0.00,0.000,0,0,No weather data available
4,2009-09-09 12:00:00,66.2,64.4,29.97,11.807,300,65,Passing clouds.


In [180]:
new_df.to_csv("weather.csv")